# Embeddings

Embeddings are a critical tool for making neural networks efficient, especially when it comes to tabular data.  Embeddings allow NN to be as powerful and tree ensemble methods for tabular data.  

When a table contains raw data (text or images) or have very high cardinality categorical features it is generally recommended to use NN instead of random forests, for example.